In [1]:
pip install requests tqdm pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 3.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import requests
import pandas as pd
from tqdm import tqdm
from urllib.parse import urlparse

# Function to download a file with resumption support and check if the download succeeded
def download_file(url, folder, expected_size, chunk_size=1024):
    # Parse the URL to get the original file name
    file_name = os.path.basename(urlparse(url).path)
    file_path = os.path.join(folder, file_name)
    
    # Check if file already exists and get its size
    if os.path.exists(file_path):
        resume_header = {'Range': f'bytes={os.path.getsize(file_path)}-'}
        resume_mode = 'ab'  # append mode
        start_pos = os.path.getsize(file_path)
    else:
        resume_header = {}
        resume_mode = 'wb'  # write mode (new file)
        start_pos = 0
    
    # Start the request with resume headers if applicable
    response = requests.get(url, stream=True, headers=resume_header)
    total_size = int(response.headers.get('content-length', 0)) + start_pos

    # Download with a progress bar
    with open(file_path, resume_mode) as file, tqdm(
        desc=file_name,
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
        initial=start_pos,
        miniters=1
    ) as bar:
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
                file.write(chunk)
                bar.update(len(chunk))
    
    # Check if the file size matches the expected size after the download
    if os.path.getsize(file_path) >= expected_size * (1024**3) * 0.99:  # Allow a small buffer for variations
        return file_name  # Return file name if download succeeded
    else:
        return None  # Return None if download is incomplete

# Read the CSV file
df = pd.read_csv('DTM_download.csv')

# Add a new 'Status' column to track the download progress if it doesn't already exist
if 'Status' not in df.columns:
    df['Status'] = 'Incomplete'

# Set the download folder (you can customize this)
download_folder = '/Users/shuyang/Data/DTM_ZIP'

# Create the download folder if it doesn't exist
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Loop through each row in the CSV and download the file
for index, row in df.iterrows():
    project = row['Project']
    package = row['Package']
    size = float(row['Size'])  # Ensure size is treated as a float (representing GB)
    resolution = row['Res. (m)']
    download_link = row['Download Link']
    
    # Check if the file is already marked as 'Completed'
    if df.loc[index, 'Status'] == 'Completed':
        print(f"{download_link} is already downloaded.")
        continue

    try:
        # Download the file and use the original name from the URL
        print(f"Downloading {package} from {project} ({size} GB)...")
        file_name = download_file(download_link, download_folder, size)
        
        # Mark as completed if the file was successfully downloaded
        if file_name:
            df.loc[index, 'Status'] = 'Completed'
            df.loc[index, 'Downloaded File Name'] = file_name
            print(f"{file_name} downloaded and verified successfully.")
        else:
            df.loc[index, 'Status'] = 'Incomplete'
            print(f"Download of {file_name} failed or incomplete.")
    
    except Exception as e:
        print(f"Error downloading {download_link}: {e}")
        df.loc[index, 'Status'] = 'Error'
        break  # Stop the loop if an error occurs

# Save the updated CSV with download status and file names
df.to_csv('DTM_download_updated.csv', index=False)
print("Download process completed. Updated CSV saved as 'DTM_download_updated.csv'.")


https://ws.gisetl.lrc.gov.on.ca/fmedatadownload/Packages/LIDAR2016to18_DTM-Crne-A.zip is already downloaded.


LIDAR2016to18_DTM-Crne-B.zip:  51%|█████     | 1.38G/2.73G [10:42<17:03, 1.41MB/s] 